In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# import libraries
from tensorflow.keras.models import load_model
import pandas as pd
import re
import string
import numpy as np

In [29]:
# import libraries and download package of nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download package nltk
nltk.download(['stopwords', 'punkt','averaged_perceptron_tagger',
               'vader_lexicon', 'wordnet', 'omw-1.4'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# load the model
model_load1=load_model('/content/drive/MyDrive/model_lstm_hotel')

In [11]:
# Create data inference
data_inf={'Property Name': 'The Rembrandt', 
          'Review Title' : 'Great Mother and Daughter Trip', 
          'Review Text' : "My mother and I stayed at the Rembrandt for 6 nights in December. We had a wonderful experience. Our room was nice and cozy. The daily breakfast was a great way to start the day. The hotel was decorated beautifully for the Christmas season. The hotel staff at the front desk, was excellent in booking my car service to and from the airport, and helping me navigate the train system, so we could take trips outside the city. The hotel was walking distance to Harrods ( which is a block away), Hyde Park, and museums. We even walked to Buckingham Palace and Big Ben. We did not use the tube, but there were multiple tube stations nearby. I would defiantly stay at this hotel again or any of it's sister properties. I had no complains. We have a lovely time.",
       'Location Of The Reviewer' : 'West Palm Beach, Florida', 'Date Of Review':'2016-02-08'}

df=pd.DataFrame([data_inf])

In [14]:
# Get the rate and review text only to be analyzed
data=df[['Review Text' ]]
data.rename({'Review Rating': 'rate', 'Review Text': 'text'}, axis=1, inplace=True)
data

<ipython-input-14-5347510b958b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename({'Review Rating': 'rate', 'Review Text': 'text'}, axis=1, inplace=True)


,text
0,My mother and I stayed at the Rembrandt for 6 ...


In [16]:
# Change to lowercase

teks = data.text[0]
teks_lower = teks.lower()
print (teks_lower)

my mother and i stayed at the rembrandt for 6 nights in december. we had a wonderful experience. our room was nice and cozy. the daily breakfast was a great way to start the day. the hotel was decorated beautifully for the christmas season. the hotel staff at the front desk, was excellent in booking my car service to and from the airport, and helping me navigate the train system, so we could take trips outside the city. the hotel was walking distance to harrods ( which is a block away), hyde park, and museums. we even walked to buckingham palace and big ben. we did not use the tube, but there were multiple tube stations nearby. i would defiantly stay at this hotel again or any of it's sister properties. i had no complains. we have a lovely time.


In [18]:
# check punctuation
punctuation = re.findall(r'[^\w\s]', teks_lower)
print (list(set(punctuation)))

['(', ')', ',', "'", '.']


In [21]:
# remove punctuation

teks_punc = teks_lower.translate(str.maketrans('', '', string.punctuation))
teks_punc

'my mother and i stayed at the rembrandt for 6 nights in december we had a wonderful experience our room was nice and cozy the daily breakfast was a great way to start the day the hotel was decorated beautifully for the christmas season the hotel staff at the front desk was excellent in booking my car service to and from the airport and helping me navigate the train system so we could take trips outside the city the hotel was walking distance to harrods  which is a block away hyde park and museums we even walked to buckingham palace and big ben we did not use the tube but there were multiple tube stations nearby i would defiantly stay at this hotel again or any of its sister properties i had no complains we have a lovely time'

In [22]:
# remove Hashtag
teks_punc = re.sub("#[A-Za-z0-9_]+", " ", teks_punc)

In [23]:
# remove \n
teks_punc = re.sub(r"\\n", " ", teks_punc)

In [24]:
# remove Whitespace
teks_punc = teks_punc.strip()

In [25]:
# Remove Emoji, Mathematic symbols (ex : μ), etc
teks_punc = re.sub("[^A-Za-z\s']", " ", teks_punc)

In [30]:
# List Stopwords
stpwds_en = list(set(stopwords.words('english')))

# tokenize words
tokens = word_tokenize(teks_punc)
teks_stopwords = [word for word in tokens if word not in stpwds_en]

In [31]:
# Define the stemmer and lemmatizer
stemmer = nltk.stem.PorterStemmer()

In [32]:
# Stemming the sentence
stemmed_words = []

for word in teks_stopwords:
    stemmed_words.append(stemmer.stem(word))
    
stemmed_sentence = " ".join(stemmed_words)
stemmed_sentence

'mother stay rembrandt night decemb wonder experi room nice cozi daili breakfast great way start day hotel decor beauti christma season hotel staff front desk excel book car servic airport help navig train system could take trip outsid citi hotel walk distanc harrod block away hyde park museum even walk buckingham palac big ben use tube multipl tube station nearbi would defiantli stay hotel sister properti complain love time'

In [42]:
# Predict the text using the model
y_pred=model_load1.predict(np.array(stemmed_sentence).reshape(-1))

1/1 [==============================] - 4s 4s/step


In [47]:
if np.argmax(y_pred)==0:
  print ("This visitor will give us rate 1-2 star rating")
elif np.argmax(y_pred)==1:
  print ("This visitor will give us rate 3 star rating")
elif np.argmax(y_pred)==2:
  print ("This visitor will give us rate 4 star rating")
elif np.argmax(y_pred)==3:
  print ("This visitor will give us rate 5 star rating")


This visitor will give us rate 5 star rating
